In [1]:
# used pretrained model to perform inference on feeny dataset

# need to take care of case when frame length > 300




In [2]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tensorflow.contrib.data import Dataset, Iterator

model_path = "/home/saurabh/Documents/tf_orange/tf_orange/models/hardik/model.ckpt"

/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)


    return mfcc_feat

In [4]:
#audio_to_mfcc('/home/saurabh/feeny_dataset/data/generated/feeny_30.wav').shape

In [5]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]

In [6]:
# we also need a fixed vocabulary 
import re

word_dictionary = {}

with open("/home/saurabh/Downloads/audio2/words.txt") as file:
    for i , line in enumerate(file):
        
        line = line.replace("\n", "")
        word_dictionary[line] = i


In [7]:
# Training Parameters
learning_rate = 0.001
training_steps = 5
batch_size = 1
display_step = 200
num_features = 13


num_classes = 493 # words.txt , all words plus space word  ( 10001) + CTC symbol (1)
timesteps = 300 # timesteps
num_hidden = 128 # hidden layer num of features

In [8]:
def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
       
        if word in word_dictionary:
           # print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list

In [9]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    #convert output shape (timesteps * batch * classes ) to (batch*timesteps*classes)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    outputs=tf.reshape(outputs, [-1,2*num_hidden] )
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    res = tf.reshape(res, [batch_size,timesteps,num_classes])

    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [10]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([ 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [11]:

inputs = tf.placeholder(tf.float32, [None, None, num_features])

seq_len = tf.placeholder(tf.int32, [None])




logits = RNN(inputs, weights, biases)




# Option 2: tf.contrib.ctc.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
decoder_input = tf.transpose(logits, [1, 0, 2])

decoded, log_prob = tf.nn.ctc_greedy_decoder(decoder_input, seq_len)


In [12]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [13]:
init = tf.global_variables_initializer()

In [14]:
# need this for decoding word give its index key value 

def keys_of_value(dct, value):
    for k in dct:
        if isinstance(dct[k], list):
            if value in dct[k]:
                return k
        else:
            if value == dct[k]:
                return k

In [15]:
keys_of_value(word_dictionary, 34)

'vehicle'

In [18]:
with tf.Session() as sess:
    

    # Run the initializer
                sess.run(init)
        
                 # Restore model weights from previously saved model
                load_path = saver.restore(sess, model_path)
              #  print("Model restored from file: %s" % save_path)
    
                
                max_len=timesteps
                    
                                 
                temp_np_inputs = np.zeros((1 , max_len , 13 ))
            
                temp_np_inputs[0]=pad(audio_to_mfcc('/home/saurabh/Downloads/audio2/Audio/7.wav'))
                    
                #test_item = np.expand_dims(, axis = 0)
                
                batch_lengths=[200]
                
                #if test_item.shape[0] < max_len:
    
                     #   difference = max_len - test_item.shape[0]
    
                      #  temp_np_inputs[0] = np.pad(test_item,((0,difference),(0,0)), mode="constant")
        
           
                print(temp_np_inputs.shape)
                print(batch_lengths)
                
            
                feed = {inputs: temp_np_inputs,
                        seq_len: batch_lengths}
	    

            
                
               
                
                # Decoding
                d = sess.run(decoded[0], feed_dict=feed)
   
                dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
    
                print(dense_decoded)
    
                str_decoded = ''.join([keys_of_value(word_dictionary, x) for x in np.asarray(d[1]) ])
    
    
                print(str_decoded)  
 
    
    
    
    

INFO:tensorflow:Restoring parameters from /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/model.ckpt
(1, 300, 13)
[200]
[[209 491  31 491  66 491 177 491 196 491]]
buying galaxies challenges current thinking 
